In [1]:
from tools import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.stats import multivariate_normal
import pandas as pd
import seaborn as sns
from scipy.linalg import fractional_matrix_power
from scipy.stats import wishart
import sys
import time

def logpdfTarget (x):
#     lnp = multivariate_normal.logpdf(x, mean=np.zeros(len(x)), cov=cov);
#     lnp = multivariate_normal.logpdf(x, mean=np.zeros(len(x)), cov=np.eye(len(x)));
    lnp = -(1/2) * (x.T).dot(x)
    return(lnp)

In [3]:
d = 100
Sigma_minus1 = wishart.rvs(df=d,scale=np.eye(d))
cov = np.linalg.inv(Sigma_minus1)
# cov = np.array([1,0.9,0.9,1]).reshape(2,2)
Sigma_neg = fractional_matrix_power(cov,-0.5)
Sigma_pos = fractional_matrix_power(cov,0.5)
# print(cov)
sum(sum(cov>0.99))/2

1494.5

In [4]:
nround = int(5*10e4)
burnin = int(10e3)
x = np.random.random(d) - 0.5
y = np.zeros(nround*d).reshape(nround,d)
accept = np.zeros(d)
sigma = np.diag(cov)
mu_star = 0
kernel = 'MirrorU'
time_begin = time.time()
random_result = (np.random.random(size = len(range(-burnin,nround))*d)-0.5) * np.sqrt(12)

for j in range(-burnin,nround):
    if (j==0):
        count = 0; accept = np.zeros(d);
    if (j % 25000 == 0):
        print ('进度中',j/25000,'/',nround/25000,' 用时: ', time.time()-time_begin)
    for i in range(d):
        initial = x.copy()
        pi_old = logpdfTarget(initial)
        if (kernel == 'Uniform'):
            initial[i] += (np.random.random()-0.5) * np.sqrt(12) * sigma    #Uniform 
        elif (kernel == 'Gaussian'):
            initial[i] += np.random.normal() * sigma                        #Gaussian
        elif (kernel == 'MirrorU'):
            NewCenter = 2 * mu_star - initial[i]
#             initial[i] = NewCenter + (np.random.random()-0.5) * np.sqrt(12) * sigma   #Mirror Uniform
            initial[i] = NewCenter + random_result[(j+burnin)*d+i] * 0.4   #Mirror Uniform
        else:
            sys.exit('No such kernel exists')
        pi_new = logpdfTarget(initial)
        pi_dif = pi_new - pi_old
        if ((pi_dif>0) | (np.exp(pi_dif)>np.random.random())):
            x = initial
            accept[i] += 1
    if (j>=0):
        y[j,:] = np.dot(Sigma_pos, x)

eff = Eff_IntegratedCorrelationTime(y[:,0])
print('sigma: ',sigma,'eff: ',eff, 'Pjump', accept[0]/nround)
# print(np.cov(y[:,0],y[:,1]))
print('协方差矩阵: \n',np.cov(np.vstack([y[:,i] for i in range(d)])))
print('总共用时: %f'% (time.time()-time_begin))

进度中 0.0 / 20.0  用时:  6.8213770389556885
进度中 1.0 / 20.0  用时:  22.586766004562378
进度中 2.0 / 20.0  用时:  38.32308077812195
进度中 3.0 / 20.0  用时:  54.08154082298279
进度中 4.0 / 20.0  用时:  70.98573207855225
进度中 5.0 / 20.0  用时:  88.21825790405273
进度中 6.0 / 20.0  用时:  104.05316877365112
进度中 7.0 / 20.0  用时:  119.8367748260498
进度中 8.0 / 20.0  用时:  135.94103503227234
进度中 9.0 / 20.0  用时:  151.82762789726257
进度中 10.0 / 20.0  用时:  167.89234566688538
进度中 11.0 / 20.0  用时:  183.6867048740387
进度中 12.0 / 20.0  用时:  199.44661498069763
进度中 13.0 / 20.0  用时:  215.20353293418884
进度中 14.0 / 20.0  用时:  230.9012689590454
进度中 15.0 / 20.0  用时:  246.68698287010193
进度中 16.0 / 20.0  用时:  262.4232029914856
进度中 17.0 / 20.0  用时:  278.23330783843994
进度中 18.0 / 20.0  用时:  293.9580841064453
进度中 19.0 / 20.0  用时:  309.69118189811707
sigma:  [ 4.21782627  1.51408905  2.51654133  6.7711047   7.4004617   1.22788115
  4.13545404  2.31440682  3.33973013  5.46037644  0.40270045 12.68605557
  2.29298075  1.9242765   3.55521436 14.60645